In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import os
import re

In [ ]:
# Загрузка данных из CSV файла 'cars_data.csv' в DataFrame
df_cars_clear = pd.read_csv('cars_data.csv')

# Преобразование столбца 'ad_id' в тип данных 'int64'
df_cars_clear['ad_id'] = df_cars_clear['ad_id'].astype('int64')

# Сортировка данных по столбцу 'parsing_time' в порядке убывания и удаление дубликатов по 'ad_id', сохраняя последние записи
df_cars_clear = df_cars_clear.sort_values('parsing_time', ascending=False).drop_duplicates('ad_id')

# Удаление строк, где есть пропуски в колонках 'name' и 'description'
df_cars_clear = df_cars_clear.dropna(subset=['name', 'description'])

# Определение, является ли объявление от автодилера, по наличию 'Автодилер' в начале значения столбца 'city'
df_cars_clear['is_dealer'] = df_cars_clear['city'].str.startswith('Автодилер', na=False)

# Преобразование столбца 'parsing_time' в тип данных datetime для удобства работы с датой и временем
df_cars_clear['parsing_time'] = pd.to_datetime(df_cars_clear['parsing_time'])


In [ ]:
# Разбиение колоники name

def name_splitting(name):
    try:
        parts = name.split(', ')
        
        if name.startswith('ВАЗ'):
            model = ' '.join(parts[0].split(' ')[:3])
            year = parts[1]
        
        
        else:
            model = ' '.join(parts[0].split(' ')[:2])
            year = parts[1]
        
        return pd.Series([model, year])
    except IndexError:
        return pd.Series([None, None])


# Разбиение колоники description 
def description_splitting(description):
    try:
        if pd.isna(description):
            return pd.Series([None] * 7)
            
        parts_description = description.split(', ')
        
        if description.startswith('Битый'):
            engine = parts_description[2].split(' ')[0]
            transmission = parts_description[2].split(' ')[1]
            power_horse = parts_description[2].split(' ')[2].strip('()')
            mileage = parts_description[1]
            car_body = parts_description[3]
            wheel_drive = parts_description[4]
            fuel_type = parts_description[5]

        elif len(parts_description) == 4:
        
            engine = parts_description[0].split(' ')[0]           
            transmission = parts_description[0].split(' ')[1]     
            power_horse = parts_description[0].split(' ')[2].strip('()')  
            mileage = 'Новый'                                     
            car_body = parts_description[1]                       
            wheel_drive = parts_description[2]                    
            fuel_type = parts_description[3]  

        elif parts_description[4] == 'электро':
            
            engine = None           
            transmission = parts_description[1].split(' ')[0]     
            power_horse = parts_description[1].split(' ')[1].strip('()')  
            mileage = parts_description[0]                                    
            car_body = parts_description[2]                       
            wheel_drive = parts_description[3]                    
            fuel_type = parts_description[4]  

    
        
        else:
            engine = parts_description[1].split(' ')[0]
            transmission = parts_description[1].split(' ')[1]
            power_horse = parts_description[1].split(' ')[2].strip('()')
            mileage = parts_description[0]
            car_body = parts_description[2]
            wheel_drive = parts_description[3]
            fuel_type = parts_description[4]
        return pd.Series([engine, transmission, mileage, power_horse, car_body, wheel_drive, fuel_type])
    except IndexError:
        return pd.Series([None] * 7)


# Функция для преобразования строки во временной интервал
def convert_to_timedelta(time_str):

    # Проверка на "Несколько секунд"
    if time_str == "Несколько секунд":
        return pd.Timedelta(seconds=2)
        
    # Используем регулярные выражения для извлечения числового значения и временной единицы
    match = re.match(r'(\d+)\s*(минуту|минуты|минут|секунд|часов|часа|час|дней|день|дня|неделю|недели)', time_str)
    if match:
        value, unit = match.groups()
        value = int(value)
        
        # Преобразуем в timedelta на основе единицы времени
        if unit in ['минуту', 'минуты', 'минут']:
            return pd.Timedelta(minutes=value)
        elif unit == 'секунд':
            return pd.Timedelta(seconds=value)
        elif unit in ['часов', 'часа', 'час']:
            return pd.Timedelta(hours=value)
        elif unit in ['дней', 'день', 'дня']:
            return pd.Timedelta(days=value)
        elif unit in ['неделю', 'недели']:
            return pd.Timedelta(weeks=value)
    return pd.NaT  # Возвращаем NaT, если строка не соответствует формату - (минуту|минуты|минут|секунд|часов|часа|час|дней|день|дня|неделю|недели)


In [ ]:
# Разделение столбца 'name' на 'model' и 'year' с использованием функции name_splitting
df_cars_clear[['model', 'year']] = df_cars_clear['name'].apply(name_splitting)

# Разделение столбца 'description' на отдельные характеристики автомобиля с использованием функции description_splitting
df_cars_clear[['engine', 'transmission', 'mileage', 'power_horse', 'car_body', 'wheel_drive', 'fuel_type']] = df_cars_clear['description'].apply(description_splitting)

# Преобразование строки 'time_update' во временной интервал
df_cars_clear['time_delta'] = df_cars_clear['time_update'].apply(convert_to_timedelta)

# Вычисление даты обновления объявления на основе времени парсинга и временного интервала
df_cars_clear['date_update'] = df_cars_clear['parsing_time'] - df_cars_clear['time_delta']

# Сброс индекса в DataFrame перед сохранением
df_cars_clear = df_cars_clear.reset_index(drop=True)

# Сохранение очищенных данных в CSV файл
df_cars_clear.to_csv('cars_data_clear.csv', index=False)

df_cars_clear.sort_values('parsing_time', ascending=False).head()